# Get Candlestick Patterns

## Import relevant libraries

In [1]:
import os
import pandas as pd
import talib
import mplfinance as mpf
import numpy as np
import matplotlib.pyplot as plt
import datetime

## Get the data

### Parameters

In [2]:
asset_ticket = "BTCUSDT"
timestamp = "1d"
start_date = "1 Jan, 2020"
end_date = "30 Sep, 2022"

In [3]:
input_data_path = '../data'
input_data_filename = "binance" + \
                  "_" + asset_ticket + \
                  "_" + timestamp + \
                  "_from_" + datetime.datetime.strptime(start_date,'%d %b, %Y').strftime('%Y_%m_%d') + \
                  "_to_" + datetime.datetime.strptime(end_date,'%d %b, %Y').strftime('%Y_%m_%d')
input_data_extension = ".csv"
full_path_input_data = os.path.join(input_data_path, input_data_filename + input_data_extension)

In [4]:
df = pd.read_csv(full_path_input_data)

In [5]:
df.columns

Index(['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time',
       'quote_asset_volumne', 'number_of_trades',
       'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore',
       'formatted_open_time', 'formatted_close_time', 'upper_shadow',
       'lower_shadow', 'real_body'],
      dtype='object')

## Identify Candlestick Patterns

In [6]:
candle_names = talib.get_function_groups()['Pattern Recognition']

- Inverted Hammer --> Single [Bullish]
- Hammer --> Single [Bullish]
- Bullish Engulfing (not available at ta-lib) --> Double [Bullish]
- Piercing Pattern --> Double [Bullish]
- Morning Star --> Triple [Bullish]


- Shooting Star --> Single [Bearish]
- Hanging Man --> Single [Bearish]
- Bearish Engulfing (not available at ta-lib) --> Double [Bearish]
- Dark Cloud Cover --> Double [Bearish]
- Evening Star --> Triple [Bearish]

- Engulfing [positive: Bullish / negative: Bearish]

In [7]:
candle_names = [
    'CDLINVERTEDHAMMER',
    'CDLHAMMER',
    'CDLPIERCING',
    'CDLMORNINGSTAR',
    'CDLSHOOTINGSTAR',
    'CDLHANGINGMAN',
    'CDLDARKCLOUDCOVER',
    'CDLEVENINGSTAR',
    'CDLENGULFING'
]

In [8]:
# extract OHLC 
op = df['open']
hi = df['high']
lo = df['low']
cl = df['close']

# create columns for each pattern
for candle in candle_names:
    # below is same as;
    # df["CDL3LINESTRIKE"] = talib.CDL3LINESTRIKE(op, hi, lo, cl)
    df[candle] = getattr(talib, candle)(op, hi, lo, cl)

In [9]:
df.sample(10)

,open_time,open,high,low,close,volume,close_time,quote_asset_volumne,number_of_trades,taker_buy_base_asset_volume,...,real_body,CDLINVERTEDHAMMER,CDLHAMMER,CDLPIERCING,CDLMORNINGSTAR,CDLSHOOTINGSTAR,CDLHANGINGMAN,CDLDARKCLOUDCOVER,CDLEVENINGSTAR,CDLENGULFING
410,1613260800000,47156.78,49707.43,47014.17,48577.79,73735.475533,1613347199999,3.584808e+09,2090070,38752.156681,...,1421.01,0,0,0,0,0,0,0,0,0
840,1650412800000,41493.19,42199.00,40820.00,41358.19,40877.350410,1650499199999,1.697039e+09,1063843,20007.296230,...,135.00,0,0,0,0,0,0,0,0,0
293,1603152000000,11751.46,12038.38,11677.59,11909.99,62134.750663,1603238399999,7.374256e+08,906111,31911.708832,...,158.53,0,0,0,0,0,0,0,0,0
397,1612137600000,33092.97,34717.27,32296.16,33526.37,82718.276882,1612223999999,2.791777e+09,1810492,41855.841486,...,433.40,0,0,0,0,0,0,0,0,0
649,1633910400000,54659.01,57839.04,54415.06,57471.35,52933.165751,1633996799999,2.994674e+09,1690409,26858.829013,...,2812.34,0,0,0,0,0,0,0,0,0
659,1634774400000,66001.40,66639.74,62000.00,62193.15,68538.645370,1634860799999,4.407758e+09,2181665,32008.514210,...,3808.25,0,0,0,0,0,0,0,0,0
78,1584576000000,5393.26,6400.00,5252.53,6162.37,199020.873439,1584662399999,1.164740e+09,1403608,99011.947011,...,769.11,0,0,0,0,0,0,0,0,0
111,1587427200000,6828.98,6940.00,6762.00,6841.37,60109.710808,1587513599999,4.118006e+08,555797,27732.164398,...,12.39,0,0,0,0,0,0,0,0,0
186,1593907200000,9135.00,9145.24,8893.03,9069.41,34073.653627,1593993599999,3.084104e+08,384500,16276.234270,...,65.59,0,100,0,0,0,0,0,0,0
784,1645574400000,38230.33,39249.93,37036.79,37250.01,43560.732000,1645660799999,1.665487e+09,1030872,21449.955800,...,980.32,0,0,0,0,0,0,0,0,0


## Split 'CDLENGULFING' into Bullish and Bearish one

In [10]:
list_open_time_bullish_engulfing = df[df['CDLENGULFING']==100]['open_time'].to_list()
list_open_time_bearish_engulfing = df[df['CDLENGULFING']==-100]['open_time'].to_list()

In [11]:
df['CDLENGULFINGBULLISH'] = np.where(df['open_time'].isin(list_open_time_bullish_engulfing), 100, 0)
df['CDLENGULFINGBEARISH'] = np.where(df['open_time'].isin(list_open_time_bearish_engulfing), -100, 0)

### Check

In [12]:
df[df['CDLENGULFING']==100].sample(5)

,open_time,open,high,low,close,volume,close_time,quote_asset_volumne,number_of_trades,taker_buy_base_asset_volume,...,CDLHAMMER,CDLPIERCING,CDLMORNINGSTAR,CDLSHOOTINGSTAR,CDLHANGINGMAN,CDLDARKCLOUDCOVER,CDLEVENINGSTAR,CDLENGULFING,CDLENGULFINGBULLISH,CDLENGULFINGBEARISH
838,1650240000000,39678.11,41116.73,38536.51,40801.13,54243.495750,1650326399999,2.153575e+09,1157741,27097.193750,...,0,0,0,0,0,0,0,100,100,0
401,1612483200000,36936.65,38310.12,36570.00,38290.24,66681.334275,1612569599999,2.509278e+09,1853253,32756.385031,...,0,0,0,0,0,0,0,100,100,0
185,1593820800000,9057.79,9190.00,9040.04,9135.46,26441.968484,1593907199999,2.405761e+08,335225,13588.819600,...,0,0,0,0,0,0,0,100,100,0
947,1659657600000,22622.41,23472.86,22586.95,23312.42,175251.697490,1659743999999,4.046436e+09,5340569,88240.554780,...,0,0,0,0,0,0,0,100,100,0
634,1632614400000,42670.63,43950.00,40750.00,43160.90,49879.997650,1632700799999,2.124657e+09,1374145,24762.744470,...,0,0,0,0,0,0,0,100,100,0


In [13]:
df[df['CDLENGULFING']==-100].sample(5)

,open_time,open,high,low,close,volume,close_time,quote_asset_volumne,number_of_trades,taker_buy_base_asset_volume,...,CDLHAMMER,CDLPIERCING,CDLMORNINGSTAR,CDLSHOOTINGSTAR,CDLHANGINGMAN,CDLDARKCLOUDCOVER,CDLEVENINGSTAR,CDLENGULFING,CDLENGULFINGBULLISH,CDLENGULFINGBEARISH
615,1630972800000,52666.20,52920.00,42843.05,46863.73,123048.802719,1631059199999,6.004106e+09,3321711,57935.601976,...,0,0,0,0,0,0,0,-100,0,-100
884,1654214400000,30452.63,30699.00,29282.36,29700.21,54067.447270,1654300799999,1.615617e+09,993769,26583.251410,...,0,0,0,0,0,0,0,-100,0,-100
271,1601251200000,10774.26,10950.00,10626.00,10696.12,50095.251734,1601337599999,5.444632e+08,686664,25219.073315,...,0,0,0,0,0,0,0,-100,0,-100
508,1621728000000,37458.51,38270.64,31111.01,34655.25,217136.046593,1621814399999,7.431242e+09,4082004,104637.659409,...,0,0,0,0,0,0,0,-100,0,-100
882,1654041600000,31801.05,31982.97,29301.00,29805.83,103395.633820,1654127999999,3.171191e+09,1404611,47614.812050,...,0,0,0,0,0,0,-100,-100,0,-100


## Export the Data

### Export parameters

In [14]:
export_path = "../data"
export_filename = "binance" + \
                  "_" + asset_ticket + \
                  "_" + timestamp + \
                  "_from_" + datetime.datetime.strptime(start_date,'%d %b, %Y').strftime('%Y_%m_%d') + \
                  "_to_" + datetime.datetime.strptime(end_date,'%d %b, %Y').strftime('%Y_%m_%d') + \
                  "_candlesticks_signals_raw"
export_extension = ".csv"
full_export_path = os.path.join(export_path, export_filename + export_extension)

### Export

In [15]:
df.to_csv(full_export_path, index=False)